In [ ]:
# %%
from pathlib import Path
import datetime as dt
import json

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import shape
from shapely.ops import transform
import pyproj

import planetary_computer
from pystac_client import Client
import stackstac
import xarray as xr
import rasterio
import rioxarray 

from dask_cuda import LocalCUDACluster
from dask.distributed import Client as DaskClient
from dask.distributed import wait

In [ ]:
# %%
AOI_GEOJSON = Path("/raid/jjayabaskar/aoi.geojson")

DATE_RANGE = ("2022-01-01", "2022-12-31")
MAX_CLOUD_FILTER = 50         # pre-filter before masking
S2_ASSETS = ["B02","B03","B04","B08","SCL"]

TARGET_RESOLUTION = 10 
STACK_CHUNKS = {"time": 1, "band": 1, "y": 2048, "x": 2048}

WORLDCOVER_COLLECTION = "io-lulc-annual-v02"
WORLDCOVER_ASSET = "data"

OUTPUT_DIR = Path("/raid/jjayabaskar/outputs/sentinel_features")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# %%
cluster = LocalCUDACluster()
client = DaskClient(cluster)
client

/raid/jjayabaskar/gis12/lib/python3.12/site-packages/distributed/scheduler.py:4070: UserWarning: Creating scratch directories is taking a surprisingly long time. (4.98s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  ServerNode.__init__(


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 503.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39525,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:41647,Total threads: 1
Dashboard: http://127.0.0.1:36211/status,Memory: 62.97 GiB
Nanny: tcp://127.0.0.1:35359,


2025-11-18 21:21:38,814 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 5cf10b96c859c64f367afbe8861192a8 initialized by task ('rechunk-merge-rechunk-transfer-172c1ff5fe25d67e6ab548d2698fe4fd', 0, 1, 3, 2, 1, 1, 3, 2) executed on worker tcp://127.0.0.1:34189
2025-11-18 21:21:38,911 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle c3d9ea30d93ac5861af9eb88929fabe3 initialized by task ('rechunk-merge-rechunk-transfer-172c1ff5fe25d67e6ab548d2698fe4fd', 0, 1, 2, 2, 1, 1, 2, 2) executed on worker tcp://127.0.0.1:37873
2025-11-18 21:21:38,946 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle aa163b1dfe57646196a80f8fe32cc8b1 initialized by task ('rechunk-merge-rechunk-transfer-172c1ff5fe25d67e6ab548d2698fe4fd', 0, 1, 0, 5, 1, 1, 0, 5) executed on worker tcp://127.0.0.1:34189
2025-11-18 21:21:39,172 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3cd9aa349e7c9de2fcffdb581a7b793b initialized by task ('rechunk-merge-rechunk-transfer-172c1ff5fe25d67e6ab5

In [ ]:
# %%
aoi_gdf = gpd.read_file(AOI_GEOJSON)
if aoi_gdf.crs is None:
    raise ValueError("AOI GeoJSON must declare a CRS.")

aoi_geom = aoi_gdf.geometry.union_all()

centroid = aoi_geom.centroid
utm_zone = int((centroid.x + 180) // 6) + 1
target_epsg = int(f"326{utm_zone:02d}") if centroid.y >= 0 else int(f"327{utm_zone:02d}")

aoi_geom_geojson = json.loads(gpd.GeoSeries([aoi_geom]).to_json())["features"][0]["geometry"]

project = pyproj.Transformer.from_crs(4326, target_epsg, always_xy=True).transform
aoi_geom_proj = transform(project, aoi_geom)
aoi_bounds = tuple(aoi_geom_proj.bounds)



In [5]:
print(aoi_bounds)
print(target_epsg)
print(aoi_geom_geojson)

(305317.1002981191, 4672613.712909488, 337897.39514006017, 4709696.839024326)
32619
{'type': 'Polygon', 'coordinates': [[[-71.35765504252703, 42.515400835973594], [-71.35765504252703, 42.18871065844891], [-70.97331384909891, 42.18871065844891], [-70.97331384909891, 42.515400835973594], [-71.35765504252703, 42.515400835973594]]]}


In [6]:
# %%
stac = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = stac.search(
    collections=["sentinel-2-l2a"],
    intersects=aoi_geom_geojson,
    datetime=f"{DATE_RANGE[0]}/{DATE_RANGE[1]}",
    query={"eo:cloud_cover": {"lt": MAX_CLOUD_FILTER}},
)

items = list(search.items())
if not items:
    raise ValueError("No Sentinel-2 scenes found for AOI/year.")

stack = (
    stackstac.stack(
        items,
        assets=S2_ASSETS,
        bounds=aoi_bounds,
        resolution=TARGET_RESOLUTION,
        epsg=target_epsg,
        chunksize=(200,1,1024,1024),
        fill_value=np.nan,
        rescale=False,
        properties=["datetime"],
    )
    .astype("float32")
    .sortby("time")
    .persist()
)

stack = stack.assign_coords(band=S2_ASSETS[: stack.sizes["band"]])
stack

<xarray.DataArray 'stackstac-14a769025d30ff5bfc56db2c30d65da6' (time: 419,
                                                                band: 5,
                                                                y: 3709, x: 3259)> Size: 101GB
dask.array<getitem, shape=(419, 5, 3709, 3259), dtype=float32, chunksize=(200, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time                 (time) datetime64[ns] 3kB 2022-01-04T15:36:41.024000...
  * band                 (band) <U3 60B 'B02' 'B03' 'B04' 'B08' 'SCL'
  * y                    (y) float64 30kB 4.71e+06 4.71e+06 ... 4.673e+06
  * x                    (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05
    id                   (time) <U54 91kB 'S2A_MSIL2A_20220104T153641_R111_T1...
    gsd                  (band) float64 40B 10.0 10.0 10.0 10.0 20.0
    title                (band) <U29 580B 'Band 2 - Blue - 10m' ... 'Scene cl...
    common_name          (band) object 40B 'blue' 'green' 'red' 'nir' None
    center_wavelength    (band) object 40B 0.49 0.56 0.665 0.842 None
    full_width_half_max  (band) object 40B 0.098 0.045 0.038 0.145 None
    epsg                 int64 8B 32619
Attributes:
    spec:        RasterSpec(epsg=32619, bounds=(305310, 4672610, 337900, 4709...
    crs:         epsg:32619
    transform:   | 10.00, 0.00, 305310.00|\n| 0.00,-10.00, 4709700.00|\n| 0.0...
    resolution:  10

In [7]:
wait([stack])

DoneAndNotDoneFutures(done={<Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 2, 3, 2, 1)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 1, 4, 3, 3)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 0, 0, 1, 2)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 2, 4, 1, 2)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 0, 1, 2, 3)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 0, 4, 2, 3)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 2, 4, 0, 0)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 2, 4, 0, 3)>, <Future: finished, type: numpy.ndarray, key: ('getitem-878c75ea27ad12101f9b68aa0ea2a613', 0, 0, 0, 0)>, <Future: finished, type: numpy.ndarr

In [ ]:
# time → date coordinate
dates = stack["time"].dt.floor("D").values
stack = stack.assign_coords(date=("time", dates))

# pull SCL out, because you probably want the SCL class with the fewest clouds
scl = stack.sel(band="SCL")
spectral = stack.drop_sel(band="SCL")

# cloud mask using SCL before the daily aggregation
clear = scl.isin([3, 4, 5, 6, 11])
spectral = spectral.where(clear, np.nan)

# collapse all scenes from the same day
daily_stack = spectral.groupby("date").median(dim="time", skipna=True)

In [9]:
daily_stack = daily_stack.chunk({"date": 10, "band": 2, "y": 1024, "x": 1024})
daily_stack

<xarray.DataArray 'stackstac-14a769025d30ff5bfc56db2c30d65da6' (date: 80,
                                                                band: 4,
                                                                y: 3709, x: 3259)> Size: 15GB
dask.array<rechunk-merge, shape=(80, 4, 3709, 3259), dtype=float32, chunksize=(10, 2, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * date     (date) datetime64[ns] 640B 2022-01-04 2022-01-22 ... 2022-12-30
  * band     (band) <U3 48B 'B02' 'B03' 'B04' 'B08'
  * y        (y) float64 30kB 4.71e+06 4.71e+06 4.71e+06 ... 4.673e+06 4.673e+06
  * x        (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05 3.379e+05
    epsg     int64 8B 32619
Attributes:
    spec:        RasterSpec(epsg=32619, bounds=(305310, 4672610, 337900, 4709...
    crs:         epsg:32619
    transform:   | 10.00, 0.00, 305310.00|\n| 0.00,-10.00, 4709700.00|\n| 0.0...
    resolution:  10

In [ ]:
# %%
daily_stack = daily_stack.assign_coords(year=("date", pd.DatetimeIndex(daily_stack["date"].values).year))
yearly_stack = daily_stack.groupby("year").median(dim="date", skipna=True)

# NDVI/NDWI/NBR
red = yearly_stack.sel(band="B04")
nir = yearly_stack.sel(band="B08")
green = yearly_stack.sel(band="B03")

feature_ds = xr.Dataset(
    {
        "bands": yearly_stack,
        "NDVI": (nir - red) / (nir + red),
        "NDWI": (green - nir) / (green + nir),
    }
)

In [12]:
feature_ds = feature_ds.chunk({"year": 1, "band": 2, "y": 1024, "x": 1024})
feature_ds

<xarray.Dataset> Size: 290MB
Dimensions:  (x: 3259, y: 3709, band: 4, year: 1)
Coordinates:
  * x        (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05 3.379e+05
  * y        (y) float64 30kB 4.71e+06 4.71e+06 4.71e+06 ... 4.673e+06 4.673e+06
  * band     (band) <U3 48B 'B02' 'B03' 'B04' 'B08'
  * year     (year) int32 4B 2022
    epsg     int64 8B 32619
Data variables:
    bands    (year, band, y, x) float32 193MB dask.array<chunksize=(1, 2, 1024, 1024), meta=np.ndarray>
    NDVI     (year, y, x) float32 48MB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    NDWI     (year, y, x) float32 48MB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>

In [13]:
# %%
label_items = list(
    stac.search(
        collections=[WORLDCOVER_COLLECTION],
        intersects=aoi_geom_geojson,
        datetime="2023-01-01/2023-12-31",
    ).items()
)
if not label_items:
    raise ValueError("No WorldCover tiles overlap the AOI/year.")

label_stack = (
    stackstac.stack(
    label_items,
    assets=[WORLDCOVER_ASSET],
    bounds=aoi_bounds,              # already in EPSG target
    resolution=TARGET_RESOLUTION,   # matches Sentinel-2 composites
    epsg=target_epsg,
    chunksize=(1,1,1024,1024),
    rescale=False,
    fill_value=np.nan,
    )
    .astype("float32")
    .persist()
)

label_stack = label_stack.assign_coords(band=["map"])
label_stack


<xarray.DataArray 'stackstac-16191f3ad2dfd3081be867d1ec48262b' (time: 2,
                                                                band: 1,
                                                                y: 3709, x: 3259)> Size: 97MB
dask.array<astype, shape=(2, 1, 3709, 3259), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * time             (time) datetime64[ns] 16B NaT NaT
  * band             (band) <U3 12B 'map'
  * y                (y) float64 30kB 4.71e+06 4.71e+06 ... 4.673e+06 4.673e+06
  * x                (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05
    id               (time) <U8 64B '19T-2023' '19T-2022'
    start_datetime   (time) <U20 160B '2023-01-01T00:00:00Z' '2022-01-01T00:0...
    ...               ...
    proj:transform   object 8B {0.0, 243910.0, 5320650.0, 10.0, -10.0}
    proj:shape       object 8B {89289, 51218}
    end_datetime     (time) <U20 160B '2024-01-01T00:00:00Z' '2023-01-01T00:0...
    io:supercell_id  (time) object 16B None '19T'
    raster:bands     object 8B {'nodata': 0, 'spatial_resolution': 10}
    epsg             int64 8B 32619
Attributes:
    spec:        RasterSpec(epsg=32619, bounds=(305310, 4672610, 337900, 4709...
    crs:         epsg:32619
    transform:   | 10.00, 0.00, 305310.00|\n| 0.00,-10.00, 4709700.00|\n| 0.0...
    resolution:  10

In [14]:
label_mosaic = stackstac.mosaic(label_stack, dim="time").squeeze("band", drop=True)

template = feature_ds["bands"].sel(year=feature_ds.year[0], band="B02")
label_mosaic = label_mosaic.rio.reproject_match(template, resampling=rasterio.enums.Resampling.nearest)

# expand to match the year dimension (only one entry now, but keeps shapes aligned)
labels_cube = xr.DataArray(
    label_mosaic.values[None, :, :],
    dims=("year", "y", "x"),
    coords={"year": feature_ds.year, "y": label_mosaic.y, "x": label_mosaic.x},
    name="worldcover",
)
labels_cube

<xarray.DataArray 'worldcover' (year: 1, y: 3709, x: 3259)> Size: 48MB
array([[[7., 7., 7., ..., 7., 7., 7.],
        [7., 7., 7., ..., 7., 7., 7.],
        [7., 7., 7., ..., 7., 7., 7.],
        ...,
        [2., 2., 2., ..., 7., 7., 7.],
        [2., 2., 2., ..., 7., 7., 7.],
        [2., 2., 2., ..., 7., 7., 7.]]],
      shape=(1, 3709, 3259), dtype=float32)
Coordinates:
  * year     (year) int32 4B 2022
  * y        (y) float64 30kB 4.71e+06 4.71e+06 4.71e+06 ... 4.673e+06 4.673e+06
  * x        (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05 3.379e+05

In [15]:
labels_cube = labels_cube.chunk({"year": 1, "y": 1024, "x": 1024})
labels_cube

<xarray.DataArray 'worldcover' (year: 1, y: 3709, x: 3259)> Size: 48MB
dask.array<xarray-<this-array>, shape=(1, 3709, 3259), dtype=float32, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int32 4B 2022
  * y        (y) float64 30kB 4.71e+06 4.71e+06 4.71e+06 ... 4.673e+06 4.673e+06
  * x        (x) float64 26kB 3.053e+05 3.053e+05 ... 3.379e+05 3.379e+05

In [ ]:
import dataclasses
from stackstac.raster_spec import RasterSpec

def raster_spec_to_plain(value):
    if isinstance(value, RasterSpec):
        if dataclasses.is_dataclass(value):
            data = dataclasses.asdict(value)
        else:
            data = {}
            for k, v in value.__dict__.items():
                if hasattr(v, "to_gdal"):           # Affine
                    data[k] = tuple(v)
                elif isinstance(v, tuple):
                    data[k] = list(v)
                else:
                    data[k] = v
        return data
    return value

for var in feature_ds.variables.values():
    var.attrs = {k: raster_spec_to_plain(v) for k, v in var.attrs.items()}

labels_ds = labels_cube.to_dataset(name="worldcover")
for var in labels_ds.variables.values():
    var.attrs = {k: raster_spec_to_plain(v) for k, v in var.attrs.items()}

In [29]:
feature_path = OUTPUT_DIR / "sentinel2_2022_annual.zarr"
labels_path = OUTPUT_DIR / "worldcover_2022_annual.zarr"

feature_future = client.compute(
    feature_ds.to_zarr(
        feature_path,
        consolidated=True,
        mode="w",
        compute=False,
    )
)

labels_future = client.compute(
    labels_cube.to_dataset(name="worldcover").to_zarr(
        labels_path,
        consolidated=True,
        mode="w",
        compute=False,
    )
)

wait([feature_future, labels_future])

/raid/jjayabaskar/gis12/lib/python3.12/site-packages/zarr/core/metadata/v3.py:297: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=3, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  d = self.to_dict()
/raid/jjayabaskar/gis12/lib/python3.12/asyncio/events.py:88: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  self._context.run(self._callback, *self._args)
/raid/jjayabaskar/gis12/lib/python3.12/site-packages/zarr/core/group.py:153: UnstableSpeci

DoneAndNotDoneFutures(done={<Future: finished, type: NoneType, key: _finalize_store-4b5b8d8b-0726-40a2-98ad-b6babd1b6fde>, <Future: finished, type: NoneType, key: _finalize_store-1106002c-6037-49f0-b5be-56ba4ce13bcc>}, not_done=set())

In [33]:
import gc
del stack, feature_ds, labels_cube
gc.collect()

NameError: name 'stack' is not defined

In [ ]:
from stackstac.raster_spec import RasterSpec

spec_dict = ds["bands"].attrs["raster_spec"]
spec = RasterSpec(**spec_dict)

In [19]:
import zarr